# Actualización del modelo
ractante por ractante (Especie por especie) de la reacción objetivo

In [1]:
import libsbml
import pandas as pd
from cobra.io import read_sbml_model
import tempfile # Creación de archivos temporales

In [2]:
# Cargar el modelo original
modelo_ori_path = '../Models/PstKB.ori.xml'
# Carga el modelo usado Cobra
modelo_ori = read_sbml_model(modelo_ori_path)
# Crear un lector de SBML
lector = libsbml.SBMLReader()
# Leer el archivo SBML y cargarlo en un objeto SBMLDocument
documento_sbml = lector.readSBML(modelo_ori_path)
# Obtener el modelo el SBML
modelo_ori_sbml = documento_sbml.getModel()

# Leer los datos del Excel
excel_path = '../Data/modelData.xls'
comparative_ofs_sheet = 'Comparative OFs'
# Solo nos interesan las últimas dos columnas ('ID' y 'PstKB_cobra'), ajusta si es necesario
comparative_ofs = pd.read_excel(excel_path, sheet_name=comparative_ofs_sheet, usecols=["ID", "PstKB_cobra"])
# Quitar [c0] o [d0] de los IDs si es necesario
comparative_ofs['ID'] = comparative_ofs['ID'].str.replace(r'\[c0\]|\[d0\]', '', regex=True)
# Convertir el DataFrame a un diccionario para fácil acceso
diccionario_coeficientes = pd.Series(comparative_ofs.PstKB_cobra.values, index=comparative_ofs.ID).to_dict()
# Nombre de las reacciones del modelo
columnas_reacciones = [reaccion.getId() for reaccion in documento_sbml.getModel().getListOfReactions()]
resultados_data = []

### Revision de generalidades del Modelo (Nombres de reacciones y Coeficientes)

In [ ]:
# Inicializar listas para almacenar los IDs de las reacciones
ids_reacciones_ori = []
ids_reacciones_ori_sbml = []
# Llenar la primera lista con los IDs de reacción del primer modelo
for reaccion in modelo_ori.reactions:
    ids_reacciones_ori.append(reaccion.id)
# Llenar la segunda lista con los IDs de reacción del segundo modelo
for reaccion in modelo_ori_sbml.reactions:
    ids_reacciones_ori_sbml.append(str(reaccion))  # Asumiendo que aquí quieres convertir el objeto reacción a string
# Crear un DataFrame con ambas listas
df_reacciones = pd.DataFrame({
    'ID_reaccion_ori': ids_reacciones_ori,
    'ID_reaccion_ori_sbml': ids_reacciones_ori_sbml
})
df_reacciones.to_csv('../Data/allReactions.csv',index=False)
df_reacciones

In [ ]:
# ID de la reacción de interés
id_reaccion = 'R_bio1_biomass'
# Obtener la reacción específica por su ID
reaccion = modelo_ori_sbml.getReaction(id_reaccion)
# Preparar una lista para almacenar los datos de los metabolitos y sus coeficientes
metabolitos_y_coeficientes = []
if reaccion:
    # Iterar sobre los reactantes y productos de la reacción
    for lista_metabolitos in [reaccion.getListOfReactants(), reaccion.getListOfProducts()]:
        for componente in lista_metabolitos:
            id_metabolito = componente.getSpecies()
            coeficiente = componente.getStoichiometry()            
            # Filtrar solo aquellos metabolitos cuyo ID contiene 'cpd'
            if 'cpd' in id_metabolito:
                metabolitos_y_coeficientes.append((id_metabolito, coeficiente))
# Convertir la lista en un DataFrame para visualización como tabla
df_metabolitos = pd.DataFrame(metabolitos_y_coeficientes, columns=['ID Metabolito', 'Coeficiente'])
# Mostrar la tabla
df_metabolitos

## One by One

In [3]:
def modificar_modelo(documento_sbml, metabolito_id, nuevo_coeficiente):
    modelo_modificado_sbml = documento_sbml.getModel()
    reaccion = modelo_modificado_sbml.getReaction('R_bio1_biomass')
    if reaccion:
        for reactante in reaccion.getListOfReactants():
            especie = modelo_modificado_sbml.getSpecies(reactante.getSpecies())
            if metabolito_id in especie.getId():
                reactante.setStoichiometry(nuevo_coeficiente)
        for producto in reaccion.getListOfProducts():
            especie = modelo_modificado_sbml.getSpecies(producto.getSpecies())
            if metabolito_id in especie.getId():
                producto.setStoichiometry(nuevo_coeficiente)
    return documento_sbml

In [4]:
resultado_inicial = modelo_ori.optimize().fluxes
# Inicializa un DataFrame vacío con los índices establecidos a los identificadores de las reacciones
df_resultados = pd.DataFrame(index=resultado_inicial.index)
df_resultados["Original"] = resultado_inicial
# Itera sobre el diccionario de coeficientes para modificar y optimizar el modelo
for metabolito_id, nuevo_coeficiente in diccionario_coeficientes.items():
    documento_modificado = modificar_modelo(documento_sbml.clone(), metabolito_id, nuevo_coeficiente)
    with tempfile.NamedTemporaryFile(suffix='.xml', delete=True) as tmp_file:
        libsbml.writeSBMLToFile(documento_modificado, tmp_file.name)
        modelo_new = read_sbml_model(tmp_file.name)
    resultado_optimizacion = modelo_new.optimize().fluxes
    # Agrega los resultados de la optimización como una nueva columna en el DataFrame
    df_resultados[metabolito_id] = resultado_optimizacion
# Ahora df_resultados tiene los ID de reacciones como índices y los ID de metabolitos como columnas
df_resultados.to_csv("../Data/OneByOne.csv")
df_resultados

,Original,cpd00001,cpd00002,cpd00003,cpd00004,cpd00005,cpd00006,cpd00008,cpd00009,cpd00010,...,cpd00834,cpd01059,cpd01080,cpd02197,cpd12844,cpd16443,cpd16503,cpd19001,cpd25914,cpd30321
rxn00001_c0,-191.417904,-397.362825,68.409652,-191.417904,-191.417904,-191.417904,-191.417904,0.0,-191.417904,-191.417904,...,-191.417904,-191.417904,-191.417904,-191.417904,-191.417904,-191.417904,-191.417904,-448.237698,-191.417904,-191.417904
rxn00001_d0,0.000000,0.000000,-274.074829,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
rxn00001_g0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
rxn00001_v0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
rxn00001_w0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EX_cpd00099_e0,-17.698141,-19.185857,-1.250508,-17.698141,-17.698141,-17.698141,-17.698141,0.0,-17.698141,-17.698141,...,-17.698141,-17.698141,-17.698141,-17.698141,-17.698141,-17.698141,-17.698141,-16.983068,-17.698141,-17.698141
EX_cpd00009_e0,-29.631576,-32.122424,-374.334715,-29.631576,-29.631576,-29.631576,-29.631576,0.0,-29.631576,-29.631576,...,-29.631576,-29.631576,-29.631576,-29.631576,-29.631576,-29.631576,-29.631576,-28.434347,-29.631576,-29.631576
EX_cpd11632_e0,-1000.000000,0.000000,0.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,0.0,-1000.000000,-1000.000000,...,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000
DM_cpd02701_c0,0.008428,0.009136,0.000595,0.008428,0.008428,0.008428,0.008428,0.0,0.008428,0.008428,...,0.008428,0.008428,0.008428,0.008428,0.008428,0.008428,0.008428,0.008087,0.008428,0.008428
